In [ ]:
from tqdm.auto import tqdm

import os
import requests
import zipfile
import cv2
import math
import matplotlib.pyplot as plt
import glob
import numpy as np
import random

from PIL import Image

In [ ]:
!pip install albumentations


In [ ]:
import os
import shutil
import random

# Klasör yolları
image_folder = "/content/drive/MyDrive/Mission3_alldata/images"
label_folder = "/content/drive/MyDrive/Mission3_alldata/labels"

# Çıkış klasörleri
output_base = "/content/drive/MyDrive/Mission3_last"
train_img_folder = os.path.join(output_base, "train/images")
train_lbl_folder = os.path.join(output_base, "train/labels")
val_img_folder = os.path.join(output_base, "val/images")
val_lbl_folder = os.path.join(output_base, "val/labels")

# Yüzdelik oran (örneğin: %80 train, %20 val)
train_ratio = 0.8

# Klasörleri oluştur
os.makedirs(train_img_folder, exist_ok=True)
os.makedirs(train_lbl_folder, exist_ok=True)
os.makedirs(val_img_folder, exist_ok=True)
os.makedirs(val_lbl_folder, exist_ok=True)

# Desteklenen resim formatları
image_extensions = {".jpg", ".png", ".jpeg"}

# Tüm resimleri ve etiketleri eşle
image_files = {f for f in os.listdir(image_folder) if os.path.splitext(f)[1] in image_extensions}
label_files = {f for f in os.listdir(label_folder) if f.endswith(".txt")}

# Eşleşen dosyaları bul
valid_files = {f for f in image_files if f.replace(os.path.splitext(f)[1], ".txt") in label_files}

# Dosyaları karıştır ve train-val olarak böl
valid_files = list(valid_files)
random.shuffle(valid_files)
split_idx = int(len(valid_files) * train_ratio)
train_files = valid_files[:split_idx]
val_files = valid_files[split_idx:]

# Train klasörüne kopyala
for file in train_files:
    shutil.copy(os.path.join(image_folder, file), os.path.join(train_img_folder, file))
    shutil.copy(os.path.join(label_folder, file.replace(os.path.splitext(file)[1], ".txt")),
                os.path.join(train_lbl_folder, file.replace(os.path.splitext(file)[1], ".txt")))

# Val klasörüne kopyala
for file in val_files:
    shutil.copy(os.path.join(image_folder, file), os.path.join(val_img_folder, file))
    shutil.copy(os.path.join(label_folder, file.replace(os.path.splitext(file)[1], ".txt")),
                os.path.join(val_lbl_folder, file.replace(os.path.splitext(file)[1], ".txt")))

print(f"Toplam {len(valid_files)} dosya bulundu.")
print(f"{len(train_files)} dosya train klasörüne, {len(val_files)} dosya val klasörüne ayrıldı.")
print("İşlem tamamlandı! ✅")


Toplam 1023 dosya bulundu.
818 dosya train klasörüne, 205 dosya val klasörüne ayrıldı.
İşlem tamamlandı! ✅


In [ ]:
import os

# Klasör yollarını belirt
image_folder = "/content/drive/MyDrive/Mission3_alldata/augmented_images"
label_folder = "/content/drive/MyDrive/Mission3_alldata/augmented_labels"

# Desteklenen resim formatları
image_extensions = {".jpg", ".png", ".jpeg"}

# Klasörlerdeki dosyaları listele
image_files = {os.path.splitext(f)[0] for f in os.listdir(image_folder) if os.path.splitext(f)[1] in image_extensions}
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_folder) if f.endswith(".txt")}

# Eksik dosyaları belirle
missing_labels = image_files - label_files  # Resmi olup etiketi olmayanlar
missing_images = label_files - image_files  # Etiketi olup resmi olmayanlar

# Sonuçları yazdır
if missing_labels:
    print("Etiketi eksik olan resimler:")
    for img in missing_labels:
        print(f"{img}.jpg veya {img}.png için .txt bulunamadı")

if missing_images:
    print("\nResmi eksik olan etiketler:")
    for lbl in missing_images:
        print(f"{lbl}.txt için görsel dosyası bulunamadı")

if not missing_labels and not missing_images:
    print("Tüm resim ve etiket dosyaları eşleşiyor! ✅")


Tüm resim ve etiket dosyaları eşleşiyor! ✅


In [1]:
!pip install albumentations

In [3]:
import albumentations as A
import cv2
import os

# 90 derece döndürme işlemi
transform = A.Compose([
    A.HorizontalFlip(p=0.66),  # Yatay çevirme
    A.Rotate(limit=359, p=1, border_mode=cv2.BORDER_CONSTANT),  # Döndürme
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

image_folder = "/content/drive/MyDrive/Mission3_alldata/images"
label_folder = "/content/drive/MyDrive/Mission3_alldata/labels"
output_folder = "/content/drive/MyDrive/Mission3_alldata/images"
output_label_folder = "/content/drive/MyDrive/Mission3_alldata/labels"

# Çıkış klasörlerini oluştur
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

for img_name in os.listdir(image_folder):
    if not (img_name.endswith(".jpg") or img_name.endswith(".png")):
        continue

    img_path = os.path.join(image_folder, img_name)
    label_path = os.path.join(label_folder, img_name.replace(".jpg", ".txt").replace(".png", ".txt"))

    print(f"Processing {img_name}...")

    # Resmi yükle
    image = cv2.imread(img_path)
    if image is None:
        print(f"Skipping {img_name}, image not loaded")
        continue

    h, w, _ = image.shape
    print(f"Image shape: {h}x{w}")

    # Mevcut etiketleri oku
    try:
        with open(label_path, 'r') as file:
            labels = file.readlines()
        print(f"Loaded labels for {img_name}")
    except FileNotFoundError:
        print(f"No label file for {img_name}, skipping")
        continue

    bboxes = []
    class_labels = []

    # Etiketleri bbox formatına dönüştür
    for label in labels:
        try:
            class_id, x_center, y_center, width, height = map(float, label.split())
            if width > 0 and height > 0:
                bboxes.append([x_center, y_center, width, height])
                class_labels.append(int(class_id))
        except ValueError:
            print(f"Skipping invalid label in {img_name}: {label}")
            continue

    if not bboxes:
        print(f"No valid bboxes for {img_name}, skipping")
        continue

    print(f"Original BBoxes: {bboxes}")

    # Augmentation uygula
    try:
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_image = transformed['image']
        aug_bboxes = transformed['bboxes']
        print(f"Augmented BBoxes: {aug_bboxes}")
    except Exception as e:
        print(f"Error in augmentation for {img_name}: {e}")
        continue

    # Geçersiz bbox'ları filtrele
    valid_bboxes = []
    valid_labels = []

    for bbox, class_id in zip(aug_bboxes, class_labels):
        x_center, y_center, width, height = bbox
        if width > 0 and height > 0:
            valid_bboxes.append(bbox)
            valid_labels.append(class_id)

    if not valid_bboxes:
        print(f"All bboxes became invalid for {img_name}, skipping")
        continue

    # Yeni dosyaları kaydet (Google Drive yerine /content içine)
    new_img_name = "0_" + img_name
    new_label_name = "0_" + img_name.replace(".jpg", ".txt").replace(".png", ".txt")

    img_save_path = os.path.join(output_folder, new_img_name)
    label_save_path = os.path.join(output_label_folder, new_label_name)

    cv2.imwrite(img_save_path, aug_image)
    print(f"Saved augmented image: {img_save_path}")

    with open(label_save_path, 'w') as f:
        for bbox, class_id in zip(valid_bboxes, valid_labels):
            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

    print(f"Saved augmented labels: {label_save_path}")

print("Augmentation completed!")


Processing circle_10.png...
Image shape: 224x224
Loaded labels for circle_10.png
Original BBoxes: [[0.257275, 0.347884, 0.345238, 0.338624]]
Augmented BBoxes: [[0.7267951347312449, 0.6749819341283371, 0.4372694288565331, 0.44118199499949545]]
Saved augmented image: /content/drive/MyDrive/Mission3_alldata/images/0_circle_10.png
Saved augmented labels: /content/drive/MyDrive/Mission3_alldata/labels/0_circle_10.txt
Processing circle_103.png...
Image shape: 224x224
Loaded labels for circle_103.png
Original BBoxes: [[0.400132, 0.425265, 0.287037, 0.287037]]
Augmented BBoxes: [[0.5326934526557247, 0.6203747215783817, 0.4029026609765476, 0.40290266097654776]]
Saved augmented image: /content/drive/MyDrive/Mission3_alldata/images/0_circle_103.png
Saved augmented labels: /content/drive/MyDrive/Mission3_alldata/labels/0_circle_103.txt
Processing circle_105.png...
Image shape: 224x224
Loaded labels for circle_105.png
Original BBoxes: [[0.60582, 0.484788, 0.531746, 0.525132]]
Augmented BBoxes: [[0.

In [ ]:
import os

folder_path = "/content/drive/MyDrive/BBR/Train/train_txt"

valid_extensions = (".jpg", ".jpeg", ".png", ".txt")

files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)])

for index, filename in enumerate(files, start=1):
    old_path = os.path.join(folder_path, filename)  # Eski dosya yolu
    ext = os.path.splitext(filename)[1]  # Dosya uzantısını al (.jpg, .txt vs.)
    new_filename = f"file_{index}{ext}"  # Yeni dosya ismi (file_1.jpg, file_2.txt vs.)
    new_path = os.path.join(folder_path, new_filename)  # Yeni dosya yolu

    os.rename(old_path, new_path)  # Dosyayı yeniden adlandır
    print(f"{filename} → {new_filename}")  # Değişiklikleri ekrana yazdır

print("\n✅ Dosya isimlendirme tamamlandı!")